In [1]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='../dataset/mnist',
                               train=True,
                               download=True,
                               transform=transform
                               )
train_loader = DataLoader(train_dataset, 
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist',
                              train=False,
                              download=True,
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10)

    def forward(self, x):
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x
    
model = Net()

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print("epoch: {}, loss: {}".format(epoch+1, running_loss/300))
            running_loss = 0.0

In [8]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('accuracy on test set: %d %%' % (100 * correct/total))


In [9]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

epoch: 1, loss: 0.599387237119178
epoch: 1, loss: 0.1846246152619521
epoch: 1, loss: 0.14400625381618737
accuracy on test set: 96 %
epoch: 2, loss: 0.11220710311084986
epoch: 2, loss: 0.10095961654869219
epoch: 2, loss: 0.08587858364917338
accuracy on test set: 97 %
epoch: 3, loss: 0.07214739452581853
epoch: 3, loss: 0.0800837468650813
epoch: 3, loss: 0.07955495078427097
accuracy on test set: 97 %
epoch: 4, loss: 0.06409845664942017
epoch: 4, loss: 0.06613188238969693
epoch: 4, loss: 0.06191968466465672
accuracy on test set: 98 %
epoch: 5, loss: 0.059337951617781076
epoch: 5, loss: 0.056646226791975396
epoch: 5, loss: 0.0549757158344922
accuracy on test set: 98 %
epoch: 6, loss: 0.05128521768686672
